## Importación de Bibliotecas y Configuración Inicial

Comenzamos importando las bibliotecas necesarias y configurando variables globales para el manejo de señales y la interrupción del algoritmo.

In [1]:
import signal
import sys
import pandas as pd
import json
import numpy as np
import time
from sklearn.model_selection import train_test_split

interrupted = False


---

## Manejo de Señales
Definimos un manejador de señales para controlar las interrupciones durante la ejecución del algoritmo.

In [2]:
def signal_handler(sig, frame):
    global interrupted
    interrupted = True

signal.signal(signal.SIGINT, signal_handler)

<function _signal.default_int_handler(signalnum, frame, /)>

---
## Carga del Conjunto de Datos

Cargamos el conjunto de datos desde un archivo CSV.

In [3]:
df = pd.read_csv('Datasets/LungCancer.csv')

---
## Exploración y Visualización de los Datos
Para entender mejor nuestros datos, podemos realizar una exploración y visualización inicial.

In [4]:
df.head()

,Sexo,Edad,Fumador,DedosAmarillos,Ansiedad,Hipertension,EnfermedadCronica,Fatiga,Alergia,Silbidos,ConsumidorAlcohol,Tos,DificultadRespirar,DificultadTragar,DolorPecho,CancerPulmon
0,M,69,0,1,1,0,0,1,0,1,1,1,1,1,1,1
1,M,74,1,0,0,0,1,1,1,0,0,0,1,1,1,1
2,F,59,0,0,0,1,0,1,0,1,0,1,1,0,1,0
3,M,63,1,1,1,0,0,0,0,0,1,0,0,1,1,0
4,F,63,0,1,0,0,0,0,0,1,0,1,1,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Sexo                284 non-null    object
 1   Edad                284 non-null    int64 
 2   Fumador             284 non-null    int64 
 3   DedosAmarillos      284 non-null    int64 
 4   Ansiedad            284 non-null    int64 
 5   Hipertension        284 non-null    int64 
 6   EnfermedadCronica   284 non-null    int64 
 7   Fatiga              284 non-null    int64 
 8   Alergia             284 non-null    int64 
 9   Silbidos            284 non-null    int64 
 10  ConsumidorAlcohol   284 non-null    int64 
 11  Tos                 284 non-null    int64 
 12  DificultadRespirar  284 non-null    int64 
 13  DificultadTragar    284 non-null    int64 
 14  DolorPecho          284 non-null    int64 
 15  CancerPulmon        284 non-null    int64 
dtypes: int64(15), object(1)
me

---
## Preprocesamiento de Datos

Definimos una función para convertir los atributos del DataFrame a variables binarias o aplicar one-hot encoding, sin modificar los atributos que ya son binarios

In [6]:
def convert_to_binary(df):
    """
    Convierte los atributos de un DataFrame a binarios o one-hot,
    sin modificar los atributos que ya son binarios.
    
    Parámetros:
    df : pd.DataFrame
        El DataFrame a convertir.

    Retorna:
    pd.DataFrame
        Un nuevo DataFrame con las columnas convertidas a formato binario o one-hot.
    """
    df_binary = df.copy()

    for column in df_binary.columns:
        # Verifica si la columna es numérica
        if df_binary[column].dtype in ['int64', 'float64']:
            if df_binary[column].nunique() == 2:
                df_binary[column] = df_binary[column].astype(int)  # No modificar columnas binarias
            elif df_binary[column].nunique() > 3:
                # Aplica binning y luego one-hot
                df_binned = pd.cut(df_binary[column], bins=4, labels=False)
                df_one_hot = pd.get_dummies(df_binned, prefix=column)
                df_binary = pd.concat([df_binary, df_one_hot], axis=1)
                df_binary.drop(column, axis=1, inplace=True)
            else:
                # Aplica one-hot directamente para columnas con más de dos valores
                df_one_hot = pd.get_dummies(df_binary[column], prefix=column)
                df_binary = pd.concat([df_binary, df_one_hot], axis=1)
                df_binary.drop(column, axis=1, inplace=True)
        elif df_binary[column].dtype == 'object':
            # Si la columna es categórica, aplica one-hot encoding
            df_one_hot = pd.get_dummies(df_binary[column], prefix=column)
            df_binary = pd.concat([df_binary, df_one_hot], axis=1)
            df_binary.drop(column, axis=1, inplace=True)

    return df_binary


In [7]:
df = convert_to_binary(df)
df.columns

Index(['Fumador', 'DedosAmarillos', 'Ansiedad', 'Hipertension',
       'EnfermedadCronica', 'Fatiga', 'Alergia', 'Silbidos',
       'ConsumidorAlcohol', 'Tos', 'DificultadRespirar', 'DificultadTragar',
       'DolorPecho', 'CancerPulmon', 'Sexo_F', 'Sexo_M', 'Edad_0', 'Edad_1',
       'Edad_2', 'Edad_3'],
      dtype='object')

---
## División de Datos en Entrenamiento y Prueba

Dividimos los datos en conjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo

In [8]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=10, stratify = Y)

print(len(Xtrain), len(Xtest))

227 57


---
## Función de Generación de Población

Definimos una función para generar la población inicial de individuos. Cada individuo consiste en una máscara y pesos normalizados.

In [9]:
def generate_population(num_individuos, num_atributos):
    """
    Genera una población inicial para el algoritmo genético.

    Parámetros:
        num_individuals (int): Número de individuos en la población.
        num_attributes (int): Número de atributos/características.

    Retorna:
        np.ndarray: Un array de población de forma (num_individuals, 2, num_attributes).
    """
    mascaras = np.random.randint(2, size=(num_individuos, num_atributos))
    pesos = np.random.uniform(-1, 1, (num_individuos, num_atributos))
    suma_pesos = pesos.sum(axis=1, keepdims=True)   
    suma_pesos[suma_pesos == 0] = 1e-10
    pesos_normalizados = pesos / suma_pesos
    poblacion = np.stack((mascaras, pesos_normalizados), axis=1)
    return poblacion

---
## Funciones de Cálculo de Aptitud

Definimos funciones para calcular la aptitud de la población y de un individuo. La aptitud se basa en la puntuación F1 o exactitud.

In [10]:
def fitness_poblacion(poblacion, Xtrain, Ytrain, umbral=0.5, metrica='f1'):
    """
    Calcula la aptitud de cada individuo en la población.

    Parámetros:
        poblacion (np.ndarray): El array de población.
        Xtrain (np.ndarray): Matriz de características de entrenamiento.
        Ytrain (np.ndarray): Etiquetas de entrenamiento.
        umbral (float): Umbral para clasificación.
        metrica (str): Métrica a utilizar para aptitud ('f1' o 'acc').

    Retorna:
        np.ndarray: Un array de puntuaciones de aptitud para la población.
    """
    poblacion = np.asarray(poblacion)
    population = np.copy(poblacion)
    population[:, 0, :] = (population[:, 0, :] > 0.5).astype(int)
    population[:, 1, :] = population[:, 1, :] * population[:, 0, :]
    sums = np.sum(population[:, 1, :], axis=1, keepdims=True)
    population[:, 1, :] = np.divide(population[:, 1, :], sums, where=sums != 0)
    probs = np.dot(Xtrain, population[:, 1, :].T)
    y_pred = (probs >= umbral).astype(int)
    y_true = Ytrain
    scores = np.empty(y_pred.shape[1])

    if metrica == 'f1':
        for i in range(y_pred.shape[1]):
            tp = np.sum((y_true == 1) & (y_pred[:, i] == 1))
            fp = np.sum((y_true == 0) & (y_pred[:, i] == 1))
            fn = np.sum((y_true == 1) & (y_pred[:, i] == 0))
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            if precision + recall == 0:
                scores[i] = 0
            else:
                scores[i] = 2 * (precision * recall) / (precision + recall)
    elif metrica == 'acc':
        for i in range(y_pred.shape[1]):
            correct_predictions = np.sum(y_true == y_pred[:, i])
            scores[i] = correct_predictions / len(y_true)

    return scores

def fitness(individuo, Xtrain, Ytrain, umbral=0.5, metrica='f1'):
    """
    Calcula la aptitud de un solo individuo.

    Parámetros:
        individual (np.ndarray): El array del individuo.
        Xtrain (np.ndarray): Matriz de características de entrenamiento.
        Ytrain (np.ndarray): Etiquetas de entrenamiento.
        umbral (float): Umbral para clasificación.
        metrica (str): Métrica a utilizar para aptitud ('f1' o 'acc').

    Retorna:
        float: La puntuación de aptitud del individuo.
    """
    individuo = np.copy(np.asarray(individuo))
    individuo[1] = individuo[1] * individuo[0] / np.sum(individuo[1] * individuo[0])
    y_true = Ytrain
    probs = np.dot(Xtrain.astype(np.float64), individuo[1, :].T.astype(np.float64))
    y_pred = (probs >= umbral).astype(int)

    if metrica == 'f1':
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        if precision + recall == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)
    elif metrica == 'acc':
        return np.sum(y_true == y_pred) / len(y_true)

---
## Función de Selección

Implementamos la selección por torneo para seleccionar individuos para la reproducción.

In [11]:
def seleccion_torneo(fitnessPoblacion, poblacion, k):
    """
    Selecciona individuos de la población utilizando selección por torneo.

    Parámetros:
        fitnessPoblacion (np.ndarray): Puntuaciones de aptitud de la población.
        poblacion (np.ndarray): El array de población.
        k (int): Tamaño del torneo.

    Retorna:
        np.ndarray: Individuos seleccionados.
    """
    seleccionados = []
    for _ in range(len(poblacion)):
        indices_torneo = np.random.choice(len(poblacion), k, replace=False)
        ganador = indices_torneo[np.argmax(fitnessPoblacion[indices_torneo])]
        seleccionados.append(poblacion[ganador])
    return np.array(seleccionados)

---
## Función de Mutación

Definimos una función para mutar individuos utilizando ruido gaussiano y pesos normalizados.

In [12]:
def mutacion_gaussiana_normalizada(individuo, prob_mutacion=0.1, sigma=0.01):
    """
    Muta un individuo utilizando ruido gaussiano y normaliza los pesos.

    Parámetros:
        individuo (np.ndarray): El individuo a mutar.
        prob_mutacion (float): Probabilidad de mutación para cada gen.
        sigma (float): Desviación estándar para el ruido gaussiano.

    Retorna:
        np.ndarray: Individuo mutado.
    """
    mutante = np.copy(individuo)
    for i in range(len(mutante[0])):
        if np.random.rand() < prob_mutacion:
            mutante[0, i] = 1 - mutante[0, i]
    for i in range(len(mutante[1])):
        if np.random.rand() < prob_mutacion:
            mutante[1, i] += np.random.normal(0, sigma)
    mutante[1] /= np.sum(np.abs(mutante[1]))
    return mutante

---
## Función de Cruce

Implementamos el cruce entre dos padres para producir dos hijos.

In [13]:
def crossover(padre1, padre2, alpha=0.5):
    """
    Realiza el cruce entre dos padres para producir dos hijos.

    Parámetros:
        padre1 (np.ndarray): El primer padre.
        padre2 (np.ndarray): El segundo padre.
        alpha (float): Parámetro que controla la mezcla de pesos.

    Retorna:
        tuple: Dos individuos hijos.
    """
    hijo1, hijo2 = np.empty(padre1.shape), np.empty(padre1.shape)
    mask = np.random.rand(len(padre1[0])) > 0.5
    hijo1[0] = np.where(mask, padre1[0], padre2[0])
    hijo2[0] = np.where(~mask, padre1[0], padre2[0])
    alphas1 = np.random.uniform(-alpha, 1 + alpha, size=len(padre1[0]))
    alphas2 = np.random.uniform(-alpha, 1 + alpha, size=len(padre1[0]))
    hijo1[1] = alphas1 * padre1[1] + (1 - alphas1) * padre2[1]
    hijo1[1] /= np.sum(np.abs(hijo1[1]))
    hijo2[1] = (1 - alphas2) * padre1[1] + alphas2 * padre2[1]
    hijo2[1] /= np.sum(np.abs(hijo2[1]))
    return hijo1, hijo2

---
## Función de Predicción

Definimos una función para hacer predicciones utilizando el mejor individuo (goat).

In [14]:
def predict(goat, Xtest):
    """
    Realiza predicciones en el conjunto de prueba utilizando el mejor individuo.

    Parámetros:
        goat (np.ndarray): El mejor individuo.
        Xtest (np.ndarray): Matriz de características de prueba.

    Retorna:
        np.ndarray: Etiquetas predichas.
    """
    goat[1] = goat[1] * goat[0] / np.sum(goat[1] * goat[0])
    probs = np.dot(Xtest.astype(np.float64), goat[1, :].T.astype(np.float64))
    y_pred = (probs >= 0.5).astype(int)
    return y_pred

---
## Funciones de Parámetros Dinámicos

Implementamos funciones para ajustar las tasas de mutación y los tamaños de torneo dinámicamente basados en el tiempo transcurrido.

In [15]:
def dynamic_mutation_rate(start_time, max_time, min_rate=0.01, max_rate=0.1):
    """
    Ajusta dinámicamente la tasa de mutación basada en el tiempo transcurrido.

    Parámetros:
        start_time (float): El tiempo de inicio del algoritmo.
        max_time (float): El tiempo máximo permitido.
        min_rate (float): La tasa mínima de mutación.
        max_rate (float): La tasa máxima de mutación.

    Retorna:
        float: Tasa de mutación ajustada.
    """
    elapsed_time = time.time() - start_time
    if elapsed_time >= max_time:
        return min_rate 
    rate = max(min_rate, max_rate * (1 - elapsed_time / max_time))
    return rate

def dynamic_tournament_size(start_time, max_time, max_size=10):
    """
    Ajusta dinámicamente el tamaño del torneo basado en el tiempo transcurrido.

    Parámetros:
        start_time (float): El tiempo de inicio del algoritmo.
        max_time (float): El tiempo máximo permitido.
        max_size (int): El tamaño máximo del torneo.

    Retorna:
        int: Tamaño de torneo ajustado.
    """
    elapsed_time = time.time() - start_time
    
    if elapsed_time >= max_time:
        return 2 
    
    size = 2 + int((elapsed_time / max_time) * (max_size - 2))  
    size = min(size, max_size) 
    
    return size

---
## Función Main

Definimos la función main que ejecuta el algoritmo genético.

In [ ]:
def main():
    """
    Función principal para ejecutar el algoritmo genético.
    """
    global interrupted


    gen_actual = 0
    max_generations = 100  
    num_individuos = 100 
    goat = np.zeros((2, Xtrain.shape[1]))
    poblacion = generate_population(num_individuos, Xtrain.shape[1])
    fit_max = 0
    rng = np.random.default_rng()
    start_time = time.time()
    total_time = 30  * 1 

    while gen_actual < max_generations and not interrupted:
        elapsed_time = time.time() - start_time 
        fitnessPob = fitness_poblacion(poblacion, Xtrain, Ytrain, umbral=0.5, metrica='f1')

        if fit_max < fitness(poblacion[np.argmax(fitnessPob)], Xtrain, Ytrain):
            goat = poblacion[np.argmax(fitnessPob)]
            fit_max = fitness(poblacion[np.argmax(fitnessPob)], Xtrain, Ytrain)

        if elapsed_time < total_time * 0.33: 
            num_elites = 1 
        elif elapsed_time < total_time * 0.66: 
            num_elites = 2 
        else: 
            num_elites = 3 

        mutation_rate = dynamic_mutation_rate(start_time, total_time)
        tournament_size = dynamic_tournament_size(start_time, total_time)

        seleccionados = seleccion_torneo(fitnessPob, poblacion, tournament_size)

        hijos = []
        indices = np.arange(len(seleccionados))
        rng.shuffle(indices)
        for i in range(0, len(seleccionados) - 1, 2):
            padre1, padre2 = indices[i], indices[i + 1]
            hijo1, hijo2 = crossover(seleccionados[padre1], seleccionados[padre2])
            hijos.append(hijo1)
            hijos.append(hijo2)

        if len(seleccionados) % 2 == 1:
            hijos.append(seleccionados[indices[-1]])

        for hijo in hijos:
            mutacion_gaussiana_normalizada(hijo, prob_mutacion=mutation_rate)

        fitness_pop_sorted_indices = np.argsort(fitnessPob)
        best_indices = fitness_pop_sorted_indices[-num_elites:]
        worst_indices = fitness_pop_sorted_indices[:num_elites]
        for i in range(num_elites):
            hijos[worst_indices[i]] = poblacion[best_indices[i]]

        poblacion = np.array(hijos)
        gen_actual += 1

        if gen_actual % 10 == 0:
            print(f"Generación {gen_actual}, Mejor aptitud: {fit_max:.4f}")

    Ypred = predict(goat, Xtest)

    print("Predicciones en el conjunto de prueba:")
    print(Ypred)

    test_fitness = fitness(goat, Xtest, Ytest)
    print(f"Aptitud en el conjunto de prueba: {test_fitness:.4f}")


In [17]:
if __name__ == "__main__":
    main()

Generación 10, Mejor aptitud: 0.7339
Generación 20, Mejor aptitud: 0.8155
Generación 30, Mejor aptitud: 0.9882
Generación 40, Mejor aptitud: 0.9882
Generación 50, Mejor aptitud: 0.9882
Generación 60, Mejor aptitud: 0.9882
Generación 70, Mejor aptitud: 0.9882
Generación 80, Mejor aptitud: 0.9882
Generación 90, Mejor aptitud: 0.9882
Generación 100, Mejor aptitud: 0.9882
Predicciones en el conjunto de prueba:
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Aptitud en el conjunto de prueba: 1.0000
